In [1]:
import numpy as np
import statsmodels.api as sm
import urllib.parse
import hashlib
import hmac
import base64
import requests
import time
import json
import logging
import websocket
import threading
import pandas as pd
import requests
import copy
import random
from IPython.display import JSON

In [ ]:
resp = requests.get(api_url+base_uri+'time')
print(resp.json())

In [ ]:
# get binanceus signature
def get_binanceus_signature(data, secret):
    postdata = urllib.parse.urlencode(data)
    message = postdata.encode()
    byte_key = bytes(secret, 'UTF-8')
    mac = hmac.new(byte_key, message, hashlib.sha256).hexdigest()
    return mac

# Attaches auth headers and returns results of a POST request
def binanceus_request_get(uri_path, data, api_key, api_sec):
    headers = {}
    headers['X-MBX-APIKEY'] = api_key
    signature = get_binanceus_signature(data, api_sec) 
    params={
        **data,
        "signature": signature,
        }           
    req = requests.get((api_url + uri_path), params=params, headers=headers)
    return req.json()

def binanceus_request_post(uri_path, data, api_key, api_sec):
    headers = {}
    headers['X-MBX-APIKEY'] = api_key
    signature = get_binanceus_signature(data, api_sec) 
    payload={
        **data,
        "signature": signature,
        }           
    req = requests.post((api_url + uri_path), headers=headers, data=payload)
    return req.text


'''
uri_path = "/api/v3/account"
data = {
    "timestamp": int(round(time.time() * 1000)),
}
'''

#get_account_result = binanceus_request(uri_path, data, api_key, secret_key)

In [ ]:
symbol='SHIBUSDT'
side='SELL'
type='LIMIT'
quantity='1000000.0'
price='0.000019'
timeInForce = 'GTC'
uri_path = "/api/v3/order"

data = {
    "symbol": symbol,
    "side": side,
    "type": type,
    "price": price,
    "quantity": quantity,
    "timeInForce": timeInForce,
    "timestamp": int(round(time.time() * 1000)) 
}

#result = binanceus_request_post(uri_path, data, api_key, secret_key)
print("POST {}: {}".format(uri_path, result))

In [ ]:

headers = {}
headers['X-MBX-APIKEY'] = api_key

resp = requests.post('https://api.binance.us/api/v3/userDataStream', headers=headers)
print(resp.json())

In [ ]:
data={
  "listenKey": 'kmYtdoP8iIsEmHn45a5vNY6E929mRpoWn8lou9UmkX8zXS2SsCovkwoaRfgh'
}

resp = requests.delete('https://api.binance.us/api/v3/userDataStream', data=data, headers=headers)

In [ ]:
listenKey = 'kmYtdoP8iIsEmHn45a5vNY6E929mRpoWn8lou9UmkX8zXS2SsCovkwoaRfgh'

In [ ]:
def close_if_below(grid, current_price, i):

        for k in range(len(df.loc[i][grid])):
            stop_price = df.loc[i][grid][0]
            
            if stop_price[0] >= current_price:

                if df.loc[i]['volume'] == df.loc[i]['volume_left']:
                    stop_volume = df.loc[i]['volume']*stop_price[1]/100
                else:
                    stop_volume = df.loc[i]['volume_left']*stop_price[1]/100



                df.at[i, 'volume_left'] = np.round(df.loc[i]['volume_left']-stop_volume, 2)
                #close() - pass price and volume 

                #calculate profit
                

                if grid == "profit_grid":
                    close_type = "Take Profit"
                    df.at[i, 'profit'] = df.at[i, 'profit']-stop_volume*(current_price-df.loc[i]['open_price'])
                else:
                    close_type = "Stop Loss"
                    df.at[i, 'profit'] = df.at[i, 'profit']+stop_volume*(current_price-df.loc[i]['open_price'])
                
                print('Type',close_type, 'Closed lot:', stop_volume, ' Price:', current_price, ' Profit:', df.at[i, 'profit'])

                del df.loc[i][grid][0]
                grid_len = len(df.loc[i][grid])
                if df.at[i, 'volume_left'] == 0.0:
                    df.at[i, 'status'] = 'closed'

def close_if_above(grid, current_price, i):
    
        for k in range(len(df.loc[i][grid])):
            stop_price = df.loc[i][grid][0]
        
            if stop_price[0] <= current_price:

                if df.loc[i]['volume'] == df.loc[i]['volume_left']:
                    stop_volume = df.loc[i]['volume']*stop_price[1]/100
                else:
                    stop_volume = df.loc[i]['volume_left']*stop_price[1]/100


                df.at[i, 'volume_left'] = np.round(df.loc[i]['volume_left']-stop_volume, 2)
                #close() - pass price and volume 

                #calculate profit
                


                if grid == "profit_grid":
                    close_type = "Take Profit"
                    df.at[i, 'profit'] = df.at[i, 'profit']+stop_volume*(current_price-df.loc[i]['open_price'])
                else:
                    close_type = "Stop Loss"
                    df.at[i, 'profit'] = df.at[i, 'profit']-stop_volume*(current_price-df.loc[i]['open_price'])
                    
                    
                print(close_type, 'Closed lot:', stop_volume, ' Price:', current_price, ' Profit:', df.at[i, 'profit'])
    
                del df.loc[i][grid][0]
                grid_len = len(df.loc[i][grid])
                if df.at[i, 'volume_left'] == 0.0:
                    df.at[i, 'status'] = 'closed'

def check_if_close(df, price):
    open_poses = df.index[df['status']=='open'].tolist()
    #pass grid type
    for i in open_poses:
        pos_type = df.loc[i]['pos_type']
        match pos_type:
                case 'long':
                    close_if_below('stop_grid', price, i)
                    close_if_above('profit_grid', price, i)
                case 'short':
                    close_if_below('profit_grid', price, i)
                    close_if_above('stop_grid', price, i)

In [2]:
# True Range and Average True Range indicator
# source_DF
#n - num of kandels, 14 by default

def indATR(source_DF,n=14):
    df = source_DF.copy()
    df['H-L']=abs(df['high']-df['low'])
    df['H-PC']=abs(df['high']-df['close'].shift(1))
    df['L-PC']=abs(df['low']-df['close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    df_temp = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df_temp


# find local mimimum / local maximum
# DF - dataframe
# i  - row in dataframe
def isLCC(DF,i):
    df=DF.copy()
    LCC=0
    if df['close'][i]<=df['close'][i+1] and df['close'][i]<=df['close'][i-1] and df['close'][i+1]>df['close'][i-1]:
        #found the local low
        LCC = i-1;
    return LCC

def isHCC(DF,i):
    df=DF.copy()
    HCC=0
    if df['close'][i]>=df['close'][i+1] and df['close'][i]>=df['close'][i-1] and df['close'][i+1]<df['close'][i-1]:
        #found the local high
        HCC = i;
    return HCC

def getMaxMinChannel(DF, n):
    maxx=0
    minn=DF['low'].max()
    for i in range (1,n):
        if maxx<DF['high'][len(DF)-i]:
            maxx=DF['high'][len(DF)-i]
        if minn>DF['low'][len(DF)-i]:
            minn=DF['low'][len(DF)-i]
    return(maxx,minn)

def check_if_signal(prepared_df):
    '''
    ohlc = get_futures_klines(symbol,100)
    prepared_df = PrepareDF(ohlc)
    '''
    
    signal=None # return value
    
    i=len(prepared_df)-1 # 99 is current kandel which is not closed, 98 is last closed candel, we need 97 to check if it is bottom or top
    
    if isLCC(prepared_df,i-1)>0:
       # found bottom - OPEN LONG
        if prepared_df['position_in_channel'][i-1]<0.5:
            # close to top of channel
            if prepared_df['slope'][i-1]<-20:
                # found a good enter point for LONG
                signal='long'

    if isHCC(prepared_df,i-1)>0:
       # found top - OPEN SHORT
        if prepared_df['position_in_channel'][i-1]>0.5:
            # close to top of channel
            if prepared_df['slope'][i-1]>20:
                # found a good enter point for SHORT
                signal='short'

    return signal, prepared_df.at[i,'close']

In [3]:
# generate data frame with all needed data:
# slope
# channel_max
# channel_min
# position_in_channel
# DF - dataframe
def PrepareDF(DF):
    ohlc = DF.iloc[:,[0,1,2,3,4]]
    ohlc.columns = ["timestamp","open","high","low","close"]
    ohlc=ohlc.set_index('timestamp')
    df = indATR(ohlc,14).reset_index() #  <====try to optimize
    df['slope'] = indSlope(df['close'],5) #  <====try to optimize
    df['channel_max'] = df['high'].rolling(10).max() #  <====try to optimize
    df['channel_min'] = df['low'].rolling(10).min() #  <====try to optimize
    df['position_in_channel'] = (df['close']-df['channel_min']) / (df['channel_max']-df['channel_min'])
    df = df.set_index('timestamp')
    df = df.reset_index()
    return(df)


# To find a slope of price line
# series - dataframe 'close' indSlope(df['close'],5)
# n - num of kandels, 5 by default

def indSlope(series,n):   
    array_sl = [j*0 for j in range(n-1)]
    for j in range(n,len(series)+1):
        y = series[j-n:j]
        x = np.array(range(n))
        x_sc = (x - x.min())/(x.max() - x.min())
        y_sc = (y - y.min())/(y.max() - y.min())
        x_sc = sm.add_constant(x_sc)
        model = sm.OLS(y_sc,x_sc)
        results = model.fit()
        array_sl.append(results.params[-1])
    slope_angle = (np.rad2deg(np.arctan(np.array(array_sl))))
    return np.array(slope_angle)

In [4]:
def print_my():
    global df
    old_df=copy.copy(df)
    while True:
        if old_df.equals(df):
            time.sleep(10)
        else:
            old_df=copy.copy(df)
            prepared_df = PrepareDF(df)
            signal, price = check_if_signal(prepared_df)
            signal = 'long'
            if signal:
                print(signal, price)
                open_pos(signal, price)
                print('deals: ',deals_array.head())

            time.sleep(10)

In [5]:
#deal with deal
# price change % // part of the lot %

#tested

def open_pos(pos_type, price, volume = 1):  #  <====try to optimize
    global profit_stop
    global deals_array
    pos_id = str(random.randrange(100000, 1000000))
    #stop_loss grid 
    #take_profit_grid
    profit_grid = []
    stop_grid = []
    match pos_type:
        case 'long':
            for i in profit_stop['take_profit']:
                profit_grid.append([price*(1+i[0]/100),i[1]/100])
            
            for k in profit_stop['stop_loss']:
                stop_grid.append([price*(1-k[0]/100),k[1]/100])            
            print(stop_grid)
            #OPEN LONG
            open_pos_binance(pos_type)
            #get pos ID
        case 'short':
            for i in profit_stop['take_profit']:
                profit_grid.append([price*(1-i[0]/100),i[1]/100])
            for k in profit_stop['stop_loss']:
                stop_grid.append([price*(1+k[0]/100),k[1]/100])
            #OPEN SHORT
            open_pos_binance(pos_type)
            #get pos ID
        case _:
            #no open poses
            pass
    long_data_df = pd.DataFrame({ 'pos_id': pos_id,
                  'pos_type': pos_type,
                  'open_price': price,
                  'volume': volume,
                  'profit_grid': [profit_grid],
                  'stop_grid': [stop_grid],
                  'status': 'opened' }, index=[0])
    
    deals_array = pd.concat([deals_array, long_data_df],
                               sort=False, ignore_index=True)


def open_pos_binance(pos_type):
    print("position opened:", pos_type)


def check_if_stop_loss():

    pass


def check_if_take_profit():
    
    active_poses = deals_array.loc[deals_array['status'] == 'open' ]
    
    match pos_type:
        case 'long':
            if current_price > take_profit[0]:
                close_pos()
                del take_profit[0]
        case 'short':
            if current_price < take_profit[0]:
                close_pos()
                del take_profit[0]
        case _:
            #no open poses
            pass
    pass
        
     
def close_pos():
    pass

In [6]:
def on_message(ws, message):
    global df
    json_message = json.loads(message)['k']
    if json_message['x'] == True:
        j=json_message
        df2 = pd.DataFrame([[j['T'], j['o'], j['h'], j['l'], j['c']]],
                           columns=['timestamp', 'open', 'high', 'low', 'close'])
        df2['timestamp'] = pd.to_datetime(j['T']//1000,unit='s')        
        df2['open']  = df2['open'].astype(float)
        df2['high']  = df2['high'].astype(float)
        df2['low']   = df2['low'].astype(float)
        df2['close'] = df2['close'].astype(float)
        print(df2)
        df.drop(index=df.index[0], 
                axis=0, 
                inplace=True)
        df = pd.concat([df, df2], sort=False, ignore_index=True)
        df.reset_index(drop=True)   

def on_close(ws):
    print("#closed#")

def stream_data(symbol, timeframe):
    socket = f'wss://stream.binance.us:9443/ws/{symbol}@kline_{timeframe}'
    ws = websocket.WebSocketApp(socket, on_message = on_message, on_close = on_close)
    ws.run_forever()

In [7]:
def get_futures_klines(symbol, timeframe, limit=500):
    global df
    global prepared_df
    prepared_df = pd.DataFrame()
    x = requests.get(f'https://api.binance.us/api/v3/klines?symbol={symbol}&limit={str(limit)}&interval={timeframe}')
    df=pd.DataFrame(x.json())
    df.drop([5,6,7,8,9,10,11], axis=1, inplace=True)
    df.rename(columns={ 0:'timestamp',
                         1:'open',
                         2:'high',
                         3:'low',
                         4:'close'}, inplace=True)
    df['timestamp'] = pd.to_datetime(df['timestamp']//1000,unit='s')
    df['open']=df['open'].astype(float)
    df['high']=df['high'].astype(float)
    df['low']=df['low'].astype(float)
    df['close']=df['close'].astype(float)
    #df['volume']=df['volume'].astype(float)

    

In [8]:
def main():
    global df
    global profit_stop
    global deals_array
    global active_positions_array
    profit_stop = {'take_profit':[[2, 20], [3, 20], [4,20], [5,20], [6,20]], 'stop_loss':[[1,100],]} #  <====try to optimize
    deals_array = pd.DataFrame(columns=
                   ['pos_id','pos_type', 'open_price', 'volume', 'profit_grid', 'stop_grid', 'status'])
    #api_key = "ThZW140lmXAIen9huJ4ycQ0JzA5dQnofoQ6eP06BsuWZSU8lnohN9IofMjnYMRXW"
    #secret_key = "MpUPVpfELbhm86qc0qPDrf8LNKEYGmZfpnsWA7HQHdiu7fK3UY50rcgXrCvMh2lx"
    api_url = "https://api.binance.us"
    symbol = 'ETHUSDT'
    limit = 50
    timeframe = '1m'
    
    
    
    get_futures_klines(symbol, timeframe, limit)
    
    
    
    t = threading.Thread(name='read stream', target=stream_data, args=(symbol.lower(),timeframe,))
    w = threading.Thread(name='print stream', target=print_my)


    w.start()
    t.start()

    w.join()
    t.join()
    



In [ ]:
main()

            timestamp     open     high      low    close
0 2022-07-04 20:06:59  1121.66  1121.66  1121.66  1121.66
long 1121.66
[[1110.4434, 1.0]]
position opened: long
deals:     pos_id pos_type open_price volume  \
0  483256     long    1121.66      1   

                                         profit_grid  status  \
0  [[1144.0932, 0.2], [1155.3098000000002, 0.2], ...  opened   

            stop_grid  
0  [[1110.4434, 1.0]]  
            timestamp     open     high      low    close
0 2022-07-04 20:07:59  1122.67  1122.67  1122.62  1122.62
long 1122.62
[[1111.3937999999998, 1.0]]
position opened: long
deals:     pos_id pos_type open_price volume  \
0  483256     long    1121.66      1   
1  429423     long    1122.62      1   

                                         profit_grid  status  \
0  [[1144.0932, 0.2], [1155.3098000000002, 0.2], ...  opened   
1  [[1145.0724, 0.2], [1156.2985999999999, 0.2], ...  opened   

                     stop_grid  
0           [[1110.4434, 1.0]]

            timestamp     open     high      low    close
0 2022-07-04 20:16:59  1124.57  1124.57  1124.57  1124.57
long 1124.57
[[1113.3243, 1.0]]
position opened: long
deals:     pos_id pos_type open_price volume  \
0  483256     long    1121.66      1   
1  429423     long    1122.62      1   
2  451572     long    1122.51      1   
3  288575     long    1122.64      1   
4  884532     long    1122.64      1   

                                         profit_grid  status  \
0  [[1144.0932, 0.2], [1155.3098000000002, 0.2], ...  opened   
1  [[1145.0724, 0.2], [1156.2985999999999, 0.2], ...  opened   
2  [[1144.9602, 0.2], [1156.1853, 0.2], [1167.410...  opened   
3  [[1145.0928000000001, 0.2], [1156.319200000000...  opened   
4  [[1145.0928000000001, 0.2], [1156.319200000000...  opened   

                     stop_grid  
0           [[1110.4434, 1.0]]  
1  [[1111.3937999999998, 1.0]]  
2  [[1111.2848999999999, 1.0]]  
3           [[1111.4136, 1.0]]  
4           [[1111.4136, 1.0]] 

In [ ]:
symbol,limit = 'SHIBUSDT', 100
x=requests.get('https://api.binance.us/api/v3/klines?symbol='+symbol+'&limit='+str(limit)+'&interval=5m')
frame=pd.DataFrame(x.json())
frame.drop([5,6,7,8,9,10,11], axis=1, inplace=True)
frame.rename(columns={ 0:'timestamp',
                     1:'open',
                     2:'high',
                     3:'low',
                     4:'close'}, inplace=True)
frame.head()

In [ ]:
a = {'t': 1656364800000, 'T': 1656365099999, 's': 'ETHUSDT', 'i': '5m', 'f': 9847556, 'L': 9847559, 'o': '1203.02000000', 'c': '1202.27000000', 'h': '1203.02000000', 'l': '1202.05000000', 'v': '1.24754000', 'n': 4, 'x': False, 'q': '1500.14277840', 'V': '0.00000000', 'Q': '0.00000000', 'B': '0'}

In [ ]:
f=pd.DataFrame(a.json())